# Single Source Shortest Path Problem

A single source problem is about finding a path between a given vertex **(called source)** to all other vertices in a graph such that the total distance between them **(source and destination)** is minimum.

The problem:
* $5$ offices in differents cities
* Travel costs between these cities are known
* Find the cheapest way from head office to branches in different cities

We have $3$ options to solve this problem
* BFS
* Dijkstra's algorithm
* Bellman Ford

### BFS for SSSP

We know that the algorithm of BFS is the following one:

```bash
enqueue any starting vertex
while queue is not empty
    p = dequeue()
    if p is unvisited
        mark it visited
        enqueue all adjacent unvisited vertices of p
```

We have one additional step to keep track of the parent node. So the complete algorithm to be used for SSSP will be

```bash
enqueue any starting vertex
while queue is not empty
    p = dequeue()
    if p is unvisited
        mark it visited
        enqueue all adjacent unvisited vertices of p
        update parent of adjacent vertices to currVertex
```

Let's create the BFS method for SSSP

In [1]:
class Graph:
    def __init__(self, graph_dict=None):
        if graph_dict is None:
            graph_dict = {}
        self.graph_dict = graph_dict

    # O(E) time complexity | O(E) time complexity
    # O(E) because in the case of SSSP we only visit
    # connected vertices so isolated vertices won't be visit
    def bfs(self, start, end):
        queue = []
        queue.append([start])
        while queue:
            path = queue.pop(0)
            node = path[-1]
            if node == end:
                return path
            for adjacent in self.graph_dict.get(node, []):
                new_path = list(path)
                new_path.append(adjacent)
                queue.append(new_path)


cust_dict = {
    'A': ['B', 'C'],
    'B': ['D', 'G'],
    'C': ['D', 'E'],
    'D': ['F'],
    'E': ['F'],
    'G': ['E']
}

graph = Graph(cust_dict)
print(graph.bfs('A', 'F'))

['A', 'B', 'D', 'F']


BFS works only for unweighted graphs.

### Why does DFS not work for SSSP?

DFS has the tendency to go ***"as far as possible"*** from the source, hence it can never find ***"shortest path"***

### Dijkstra's algorithm for SSSP

The first thing we need to do is to create our graph class, but this time it will be a weighted graphh class

In [2]:
from collections import defaultdict

# The graph class and the main funtions needed for Dijkstra's algorithm
class Graph:
    def __init__(self):
        self.nodes = set()
        self.edges = defaultdict(list)
        self.distances = {}

    def add_node(self, value):
        self.nodes.add(value)


    def add_edge(self, from_node, to_node, distance):
        self.edges[from_node].append(to_node)
        self.distances[(from_node, to_node)] = distance


# O(V^2) time complexity (2 nested loops) | O(E) space complexity cause we're storing edges
def dijkstra(graph, initial):
    visited = {initial : 0}
    path = defaultdict(list)

    nodes = set(graph.nodes)

    while nodes:
        min_node = None
        for node in nodes:
            if node in visited:
                if min_node is None:
                    min_node = node

        if min_node is None:
            break

        nodes.remove(min_node)
        current_weight = visited[min_node]

        for edge in graph.edges[min_node]:
            weight = current_weight + graph.distances[(min_node, edge)]
            if edge not in visited or weight < visited[edge]:
                visited[edge] = weight
                path[edge].append(min_node)

    return visited, path

In [3]:
custum_graph = Graph()
custum_graph.add_node("A")
custum_graph.add_node("B")
custum_graph.add_node("C")
custum_graph.add_node("D")
custum_graph.add_node("E")
custum_graph.add_node("F")
custum_graph.add_node("G")

custum_graph.add_edge("A", "B", 2)
custum_graph.add_edge("A", "C", 5)
custum_graph.add_edge("B", "C", 6)
custum_graph.add_edge("B", "D", 1)
custum_graph.add_edge("B", "E", 3)
custum_graph.add_edge("C", "F", 8)
custum_graph.add_edge("D", "E", 4)
custum_graph.add_edge("E", "G", 9)
custum_graph.add_edge("F", "G", 7)

print(dijkstra(custum_graph, "A"))

({'A': 0, 'B': 2, 'C': 5, 'D': 3, 'E': 5, 'F': 13, 'G': 14}, defaultdict(<class 'list'>, {'B': ['A'], 'C': ['A'], 'D': ['B'], 'E': ['B'], 'F': ['C'], 'G': ['F', 'E']}))


### Bellman Ford Algorithm ofr SSSP

Here is a summary of the efficiency of the different graph algorithms

|            Graph Type            |   BFS    | Dijkstra | Bellman Ford |
| :------------------------------: | :------: | :------: | :----------: |
|     Unweighted - Undirected      |    OK    |    OK    |      OK      |
|      Unweighted - Directed       |    OK    |    OK    |      OK      |
| Positive - Weighted - Undirected | $\times$ |    OK    |      OK      |
|  Positive - Weighted - Directed  | $\times$ |    OK    |      OK      |
| Negative - Weighted - Undirected | $\times$ |    OK    |      OK      |
|  Negative - Weighted - Directed  | $\times$ |    OK    |      OK      |
|          Negative cycle          | $\times$ | $\times$ |      OK      |


Bellman Ford algorithm is used to find SSSP problem, if there is a negative cycle it catches it and report its existence.

It works basically the same way as Dijkstra's one but with small differences

```bash
if the distance of destination vertex > (distance of source vertex + weight between source and destination):
    Update distance of destination vertex to (distance of source vertex + weight between source and destination vertex)
```

In [7]:
class Graph:
    def __init__(self, vertices):
        self.vertice = vertices
        self.graph = []
        self.nodes = []


    def add_edge(self, source, destination, weight):
        self.graph.append([source, destination, weight])

    def add_node(self, value):
        self.nodes.append(value)

    def print_solution(self, distance):
        print("Vertex distance from source")
        for key, value in distance.items():
            print(' ' + key, ' : ', value)
    
    # O(EV) time complexity | O(V) space complexity cause we create a dictionnary for distances
    def bellman_ford(self, source):
        distance = {i : float("Inf") for i in self.nodes} # O(V) time complexity
        distance[source] = 0

        for _ in range(self.vertice-1): # O(V) time complexity
            for source, destination, weight in self.graph: # O(E) time complexity
                if distance[source] != float("Inf") and distance[source] + weight < distance[destination]:
                    distance[destination] = distance[source] + weight

        for source, destination, weight in self.graph: # O(E) time complexity
            if distance[source] != float("Inf") and distance[source] + weight < distance[destination]:
                print("Graph contains negative cycle")
                return

        self.print_solution(distance)

In [8]:
custum_graph = Graph(5)
custum_graph.add_node("A")
custum_graph.add_node("B")
custum_graph.add_node("C")
custum_graph.add_node("D")
custum_graph.add_node("E")

custum_graph.add_edge("A", "C", 6)
custum_graph.add_edge("A", "D", 6)
custum_graph.add_edge("B", "A", 3)
custum_graph.add_edge("C", "D", 1)
custum_graph.add_edge("D", "C", 2)
custum_graph.add_edge("D", "B", 1)
custum_graph.add_edge("E", "B", 4)
custum_graph.add_edge("E", "D", 2)

custum_graph.bellman_ford("E")

Vertex distance from source
 A  :  6
 B  :  3
 C  :  4
 D  :  2
 E  :  0


It takes :

* 6 unites form E to A
* 3 units from E to B
* $\dots$

### Conclustion

|    Graph Type    |                                             BFS                                             |                                        Dijkstra                                        |                           Bellman Ford                            |
| :--------------: | :-----------------------------------------------------------------------------------------: | :------------------------------------------------------------------------------------: | :---------------------------------------------------------------: |
| Time complexity  |                                          $O(V^2)$                                           |                                        $O(V^2)$                                        |                              $O(VE)$                              |
| Space complexity |                                           $O(E)$                                            |                                         $O(V)$                                         |                              $O(V)$                               |
|  Implementation  |                                            Easy                                             |                                        Moderate                                        |                             Moderate                              |
|    Limitation    |                               Doesn't work for weighted graph                               |                            Doesn't work for negative cycle                             |                                N/A                                |
| Unweighted graph | <t style="background : green">use this as time complexity is good and easy to implement</t> |                          Don't use as implementation not easy                          |                 Not use as time complexity is bad                 |
|  Weighted graph  |                                        Not supported                                        | <t style="background : green">use this as time complexity is better than Bellman's</t> |               Dont't use as time complexity is bad                |
|  Negative cycle  |                                        Not supperted                                        |                                     Not supported                                      | <t style="background : green">use this as others don't support/t> |
